Test shipping

In [5]:
import sys
import os
import json
import logging
from kafka import KafkaConsumer, KafkaProducer
from json import loads
import random
from datetime import datetime,timedelta
import mysql.connector
from logging.handlers import RotatingFileHandler

In [6]:
def shipping_consumer():
    # MySQL connection
    mysql_conn = mysql.connector.connect(
        host='localhost',
        user='mysql',
        password='mysql',
        database='oaken'
    )
    mysql_cursor = mysql_conn.cursor()

    # Create a consumer instance
    sales_consumer = KafkaConsumer(
        'invoices',
        bootstrap_servers=['localhost:9092','localhost:9093','localhost:9094'],
        auto_offset_reset='earliest',  # Start consuming from the earliest offset
        enable_auto_commit=True,       # Automatically commit offsets
        group_id='shipping_group',  # Specify a consumer group
        value_deserializer=lambda x: loads(x.decode('utf-8')))

    sales_consumer.subscribe(topics=['mysql'])

    shipping_producer = KafkaProducer(bootstrap_servers=['localhost:9092','localhost:9093','localhost:9094'],
                         value_serializer=lambda x: json.dumps(x).encode('utf-8'))

    sales_consumer.subscribe(topics='invoices')

    shipping_producer = KafkaProducer(bootstrap_servers=['localhost:9092'],
                            value_serializer=lambda x: json.dumps(x).encode('utf-8'))

    print('Set up complete')

    # Poll for messages
    try:
        for message in sales_consumer:
            try:
                data = message.value
                invoice = data.get('Invoice', '')

                date_str = data.get('SaleDate','')
                sales_date = datetime.strptime(date_str, '%Y-%m-%d').date()

                sales = data.get('saleDollars','')

                shipping_cost = round(float(sales) * 0.05,2)
                shipping_cost_str = str(shipping_cost)

                random_days = random.randint(0, 4)
                shipping_date = sales_date + datetime.timedelta(days=random_days)
                shipping_date_str = str(shipping_date)

                sales_consumer.commit()
                # MySQL
                UPDATE_QUERY = """
                    UPDATE sales
                    SET ShippingDate = %s, ShippingCost = %s
                    WHERE Invoice = %s
                """


                update_data = (shipping_date, shipping_cost, invoice)
                mysql_cursor.execute(UPDATE_QUERY, update_data)

                mysql_conn.commit()
            except Exception as e:
                print(f"Error processing message: {e}")

            try:
                # Kafka topic
                shipping_info = {
                    'Invoice': invoice,
                    'SalesDate': date_str,
                    'SaleDollars': sales,
                    'ShippingDate': shipping_date_str,
                    'ShippingCost': shipping_cost_str
                }

                shipping_producer.send('shipping', value=shipping_info)
                shipping_producer.flush()
            except Exception as e:
                print(f"Error processing message: {e}")
                pass

    # Close MySQL connection
    finally:
        mysql_cursor.close()
        mysql_conn.close()


In [7]:
shipping_consumer()

Set up complete


KeyboardInterrupt: 

In [ ]:
# sample output
'''
{'Invoice/Item Number': 'S29198800001', 'Date': '11/20/2015', 'Store Number': '2191', 'Store Name': 'Keokuk Spirits', 
'Address': '1013 MAIN', 'City': 'KEOKUK', 'Zip Code': '52632', 'Store Location': '1013 MAIN\nKEOKUK 52632\n(40.39978, -91.387531)', 
'County Number': '56', 'County': 'Lee', 'Category': '', 'Category Name': '', 'Vendor Number': '255', 'Vendor Name': 'Wilson Daniels Ltd.', 
'Item Number': '297', 'Item Description': 'Templeton Rye w/Flask', 'Pack': '6', 'Bottle Volume (ml)': '750', 'State Bottle Cost': '$18.09', 
'State Bottle Retail': '$27.14', 'Bottles Sold': '6', 'Sale (Dollars)': '$162.84', 'Volume Sold (Liters)': '4.50', 'Volume Sold (Gallons)': '1.19'}
'''